In [1]:
import os
import pickle
import torch
import numpy as np
import torch.nn as nn
from PIL import Image
from torchvision import models, transforms,datasets

print(torch.__version__) 

1.11.0


In [2]:
# Check availability of GPU

use_gpu = torch.cuda.is_available()
# use_gpu = False # Uncomment in case of GPU memory error
if use_gpu:
    print('GPU is available!')
    device = "cuda"
else:
    print('GPU is not available!')
    device = "cpu"

GPU is available!


In [3]:
# Load train-test list
with open('trainList_5class.pckl','rb') as f:
    trainList = pickle.load(f)
with open('testList_5class.pckl','rb') as f:
    testList = pickle.load(f)

In [5]:
trainList

['v_ApplyEyeMakeup_g01_c01',
 'v_ApplyEyeMakeup_g01_c02',
 'v_ApplyEyeMakeup_g01_c03',
 'v_ApplyEyeMakeup_g01_c04',
 'v_ApplyEyeMakeup_g01_c05',
 'v_ApplyEyeMakeup_g01_c06',
 'v_ApplyEyeMakeup_g02_c01',
 'v_ApplyEyeMakeup_g02_c02',
 'v_ApplyEyeMakeup_g02_c03',
 'v_ApplyEyeMakeup_g02_c04',
 'v_ApplyEyeMakeup_g03_c01',
 'v_ApplyEyeMakeup_g03_c02',
 'v_ApplyEyeMakeup_g03_c03',
 'v_ApplyEyeMakeup_g03_c04',
 'v_ApplyEyeMakeup_g03_c05',
 'v_ApplyEyeMakeup_g03_c06',
 'v_ApplyEyeMakeup_g04_c01',
 'v_ApplyEyeMakeup_g04_c02',
 'v_ApplyEyeMakeup_g04_c03',
 'v_ApplyEyeMakeup_g04_c04',
 'v_ApplyEyeMakeup_g04_c05',
 'v_ApplyEyeMakeup_g04_c06',
 'v_ApplyEyeMakeup_g04_c07',
 'v_ApplyEyeMakeup_g05_c01',
 'v_ApplyEyeMakeup_g05_c02',
 'v_ApplyEyeMakeup_g05_c03',
 'v_ApplyEyeMakeup_g05_c04',
 'v_ApplyEyeMakeup_g05_c05',
 'v_ApplyEyeMakeup_g05_c06',
 'v_ApplyEyeMakeup_g05_c07',
 'v_ApplyEyeMakeup_g06_c01',
 'v_ApplyEyeMakeup_g06_c02',
 'v_ApplyEyeMakeup_g06_c03',
 'v_ApplyEyeMakeup_g06_c04',
 'v_ApplyEyeMa

In [6]:
classes = []
for item in trainList:
    c = item.split('_')[1]
    if c not in classes:
        classes.append(c)
print(classes)

['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam']


### Initialize network and load trained weights

In [7]:
net = models.resnet18()
net.fc = nn.Linear(512,5)
# Loading saved states
net.load_state_dict(torch.load('resnet18Pre_fcOnly5class_ucf101_10adam_1e-4_b128.pt'))
# Removing fully connected layer for feature extraction
model = nn.Sequential(*list(net.children())[:-1]).to(device)

### Feature extraction

In [8]:
# PIL image to tensor transformation
data_transforms = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),            
        transforms.ToTensor()
    ])

In [9]:
framePath = 'frames/'
for item in trainList:
    cName = item.split('_')[1]
    srcPath = framePath+cName+'/'+item    
    fNames = os.listdir(srcPath)
    # filename template
    fTemplate = fNames[0].split('_')
    fCount = len(fNames)
    for fNum in range(fCount):
        fileName = fNames[fNum]
        if os.path.exists(srcPath+'/'+fileName):
            # Loading image
            img = Image.open(srcPath+'/'+fileName)
            # Transform to tensor
            imgTensor = data_transforms(img).unsqueeze(0)
            inp = imgTensor.to(device)
            # Feed-forward through model+stack features for each video
            if fNum == 0:
                out = model(inp)               
                if use_gpu:
                    out = out.cpu()
                out = out.view(out.size()[0],-1).data             
            else:
                out1 = model(inp)               
                if use_gpu:
                    out1 = out1.cpu()
                out1 = out1.view(out1.size()[0],-1).data.cpu()                
                out = torch.cat((out,out1),0)
        else:
            print(fileName+ ' missing!')       
    # out dimension -> frame count x 512
    featSavePath = 'ucf101_resnet18Feat/train/'+cName # Directory for saving features
    if not os.path.exists(featSavePath):
        os.makedirs(featSavePath)
    torch.save(out,os.path.join(featSavePath,item+'.pt'))   
    

In [11]:
framePath = 'frames/'
for item in testList:
    cName = item.split('_')[1]
    srcPath = framePath+cName+'/'+item    
    fNames = os.listdir(srcPath)
    fTemplate = fNames[0].split('_')
    fCount = len(fNames)
    for fNum in range(fCount):
        fileName = fNames[fNum]
        if os.path.exists(srcPath+'/'+fileName):
            img = Image.open(srcPath+'/'+fileName)
            imgTensor = data_transforms(img).unsqueeze(0)
            inp = imgTensor.to(device)
            if fNum == 0:
                out = model(inp) 
                if use_gpu:
                    out = out.cpu()
                out = out.view(out.size()[0],-1).data               
            else:
                out1 = model(inp) 
                if use_gpu:
                    out1 = out1.cpu()
                out1 = out1.view(out1.size()[0],-1).data                
                out = torch.cat((out,out1),0)
        else:
            print(fileName+ ' missing!')
      
    featSavePath = 'ucf101_resnet18Feat/test/'+cName
    if not os.path.exists(featSavePath):
        os.makedirs(featSavePath)
    torch.save(out,os.path.join(featSavePath,item+'.pt'))   